In [19]:
import pandas as pd

In [20]:
czk_df = pd.read_parquet('data/FRED_Req.parquet')
covid_df = pd.read_parquet('data/OWID_Req.parquet')

In [21]:
#make df from FRED data, convert index to datetime
czk_df.index = pd.to_datetime(czk_df.index)
czk_df = czk_df.loc['2020-03-01':,:] #this does work but with warning
czk_df.columns = ['CZK/USD']
czk_df['CZK/USD'] = czk_df['CZK/USD'].astype(float)
#df

In [22]:
#cut data on Czech Republic, drop empty columns and rows, convert index to datetime
covid_df = covid_df.where(covid_df['location'] == 'Czechia').dropna(how='all')
covid_df = covid_df.dropna(axis=1, how='all')
covid_df.index = pd.to_datetime(covid_df.index)
#covid_df

In [23]:
#problem: FRED data monthly, OWID data daily -> need to convert OWID data
#add up cases of a month, save in a list
list_monthly_cases = []
list_monthly_cases_index = []
monthly_cases = covid_df.iloc[0]['new_cases']
for i in range(len(covid_df)-1):
    if covid_df.index[i].month == covid_df.index[i+1].month:
        monthly_cases = monthly_cases + covid_df.iloc[i+1]['new_cases']
    else:
        list_monthly_cases.append(monthly_cases)
        list_monthly_cases_index.append(covid_df.index[i+1])
        monthly_cases = covid_df.iloc[i+1]['new_cases']

In [24]:
'''
#control if idea works -> it does
print(list_monthly_cases)
print(covid_df.loc['2020-04-30']['total_cases']-covid_df.loc['2020-03-31']['total_cases'])
covid_df.loc['2020-03-31']['total_cases']
'''

"\n#control if idea works -> it does\nprint(list_monthly_cases)\nprint(covid_df.loc['2020-04-30']['total_cases']-covid_df.loc['2020-03-31']['total_cases'])\ncovid_df.loc['2020-03-31']['total_cases']\n"

In [25]:
#safe list of monthly cases as df, concat with df on CZK/USD data
monthly_cases_summed = pd.DataFrame(data = list_monthly_cases, index = list_monthly_cases_index, columns = ['Summed Monthly Cases'])
result = pd.concat([czk_df,monthly_cases_summed], axis = 1)
result

,CZK/USD,Summed Monthly Cases
2020-03-01,24.042864,NaN
2020-04-01,25.069682,3308.0
2020-05-01,25.012381,4374.0
2020-06-01,23.706455,1586.0
2020-07-01,23.145000,2686.0
2020-08-01,22.122381,4620.0
2020-09-01,22.679909,8044.0
2020-10-01,23.098409,46145.0
2020-11-01,22.357048,264339.0
2020-12-01,21.629913,188196.0


In [26]:
result.to_parquet('data/merged_data.parquet')